In [1]:
def prepare_dir(output_dir, chunk_df, template, params):

    os.makedirs(output_dir, exist_ok=True)
    cell_table_path = f'{output_dir}cell_table.csv'
    chunk_df.to_csv(cell_table_path, header=False, index=True)
    params_str = '\n'.join(f'{k} = {v}' for k, v in params.items())

    with open(f'{output_dir}Snakefile_master', 'w') as f:
        f.write(params_str + template)
    return

In [2]:
params = {
    'resolution': 25000,
    'chrom_size_path': '"/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes"',
}


In [4]:
# cell_table = pd.concat([pd.read_csv(path, sep = '\t', header = None).set_index(0) 
#                         for path in glob(f"cell_table/*")])

In [5]:
import pandas as pd
cell_table = pd.read_csv('All.25K.ImputedCool.csv').set_index('cell_id')[['file_paths']]

In [6]:
ct = 'Oligo_NN'

In [7]:
use_cts = joblib.load('/data/metadata/m3c_use_cts')
use_cts = [ct.replace(' ','_').replace('/','') for ct in use_cts]

meta = pd.read_csv('/data/metadata/240104_m3C_META.csv', index_col = 0)
meta['AgingMajorType'] = meta['AgingMajorType'].apply(lambda x: x.replace(' ','_').replace('/',''))
meta = meta[meta['AgingMajorType'].isin(use_cts)]
meta['AgingMajorType.Age'] = meta['AgingMajorType'] + '.' + meta['Age']

In [8]:
meta = meta[meta['AgingMajorType'] == ct]

In [9]:
cell_table['cluster'] = meta['AgingMajorType.Age'].copy()
cell_table = cell_table.dropna()

In [11]:
chunk_size = 200
res = 25000
total_chunk_dirs = []
group_chunks = {}

In [12]:
with open(f'{PACKAGE_DIR}/cool/Snakefile_chunk_template') as tmp:
    GENERATE_MATRIX_CHUNK_TEMPLATE = tmp.read()

In [13]:
outdir = '/home/qzeng_salk_edu/project/240205-domain/dataset/hicluster_bulk/'

In [14]:
for group, group_df in cell_table.groupby('cluster'):
    group_chunks[group] = []
    if group_df.shape[0] <= chunk_size:
        this_dir = f'{outdir}{group}_chunk0/'
        params['cell_table_path'] = f'"{this_dir}cell_table.csv"'
        prepare_dir(this_dir, group_df, GENERATE_MATRIX_CHUNK_TEMPLATE, params)
        total_chunk_dirs.append(this_dir)
        group_chunks[group].append(this_dir)
    else:
        group_df['chunk'] = [i // chunk_size for i in range(group_df.shape[0])]
        for chunk, chunk_df in group_df.groupby('chunk'):
            this_dir = f'{outdir}{group}_chunk{chunk}/'
            params['cell_table_path'] = f'"{this_dir}cell_table.csv"'
            prepare_dir(this_dir, chunk_df, GENERATE_MATRIX_CHUNK_TEMPLATE, params)
            total_chunk_dirs.append(this_dir)
            group_chunks[group].append(this_dir)

            

In [15]:
with open(f'{outdir}snakemake_cmd_step1.txt', 'w') as f:
    for chunk_dir in total_chunk_dirs:
        cmd = f'snakemake -d {chunk_dir} --snakefile {chunk_dir}Snakefile_master -j 5 --rerun-incomplete'
        f.write(cmd + '\n')

In [16]:
params.pop('cell_table_path')
params_str = '\n'.join(f'{k} = {v}' for k, v in params.items())

with open(f'{PACKAGE_DIR}/cool/Snakefile_group_template') as tmp:
    GENERATE_MATRIX_GROUP_TEMPLATE = tmp.read()

with open(f'{outdir}Snakefile', 'w') as f:
    f.write(params_str + '\n' + GENERATE_MATRIX_GROUP_TEMPLATE)
    
with open(f'{outdir}snakemake_cmd_step2.txt', 'w') as f:
    cmd = f'snakemake -d {outdir} --snakefile {outdir}Snakefile -j 10 --rerun-incomplete'
    f.write(cmd + '\n')